# **Understanding the Data Science Job Market**

### **Setting the scene**

In this notebook, we investigate a data set of web scraped data science jobs from Glassdoor in February 2021, compiled by [Rashik Rahman](https://www.kaggle.com/rashikrahmanpritom), found on Kaggle [here](https://www.kaggle.com/rashikrahmanpritom/data-science-job-posting-on-glassdoor).
We initially do some high-level exploratory analysis, followed by a brief attempt to see whether a linear regression model can shed some light on the data science job market.

---

Summary:

*   Posted salary does not correlate with size of the company or revenue generated
*   List item

---

### **Importing required libraries and data**

First we import the libraries that we might need and take a glimpse at the data

In [4]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
from matplotlib.ticker import FormatStrFormatter
import seaborn as sns
import plotly.express as px

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error

import random

import json
%matplotlib inline

In [5]:
random.seed(0)
pd.set_option('display.max_columns', None)

import warnings
warnings.filterwarnings('ignore')

In [ ]:
!wget --no-check-certificate \
    'https://docs.google.com/spreadsheets/d/e/2PACX-1vQQPg9_DhCbQuz2t2EXJoiCgeOSA-0-jKZ6zk0XyBQlzFQMDFjdQILRPoNTm9K8l2xHS44Ke9KK_KUp/pub?gid=303885414&single=true&output=csv' \
    -O /tmp/dataset.csv

scraped_jobs_data = pd.read_csv('/tmp/dataset.csv', sep=',')

In [8]:
scraped_jobs_data.head()

,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,Type of ownership,Industry,Sector,Revenue,min_salary,max_salary,avg_salary,job_state,same_state,company_age,python,excel,hadoop,spark,aws,tableau,big_data,job_simp,seniority
0,Sr Data Scientist,137-171,Description\n\nThe Senior Data Scientist is re...,3.1,Healthfirst,"New York, NY","New York, NY",1001 to 5000 employees,Nonprofit Organization,Insurance Carriers,Insurance,Unknown / Non-Applicable,137,171,154,NY,1,27,0,0,0,0,1,0,0,data scientist,senior
1,Data Scientist,137-171,"Secure our Nation, Ignite your Future\n\nJoin ...",4.2,ManTech,"Chantilly, VA","Herndon, VA",5001 to 10000 employees,Company - Public,Research & Development,Business Services,$1 to $2 billion (USD),137,171,154,VA,0,52,0,0,1,0,0,0,1,data scientist,na
2,Data Scientist,137-171,Overview\n\n\nAnalysis Group is one of the lar...,3.8,Analysis Group,"Boston, MA","Boston, MA",1001 to 5000 employees,Private Practice / Firm,Consulting,Business Services,$100 to $500 million (USD),137,171,154,MA,1,39,1,1,0,0,1,0,0,data scientist,na
3,Data Scientist,137-171,JOB DESCRIPTION:\n\nDo you have a passion for ...,3.5,INFICON,"Newton, MA","Bad Ragaz, Switzerland",501 to 1000 employees,Company - Public,Electrical & Electronic Manufacturing,Manufacturing,$100 to $500 million (USD),137,171,154,MA,0,20,1,1,0,0,1,0,0,data scientist,na
4,Data Scientist,137-171,Data Scientist\nAffinity Solutions / Marketing...,2.9,Affinity Solutions,"New York, NY","New York, NY",51 to 200 employees,Company - Private,Advertising & Marketing,Business Services,Unknown / Non-Applicable,137,171,154,NY,1,22,1,1,0,0,0,0,0,data scientist,na


In [9]:
scraped_jobs_data.describe()

,Rating,min_salary,max_salary,avg_salary,same_state,company_age,python,excel,hadoop,spark,aws,tableau,big_data
count,660.000000,660.000000,660.000000,660.000000,660.000000,660.000000,660.000000,660.000000,660.000000,660.000000,660.000000,660.000000,660.000000
mean,3.587424,99.296970,148.301515,123.612121,0.407576,29.736364,0.730303,0.440909,0.212121,0.281818,0.260606,0.184848,0.206061
std,1.183540,33.161485,48.264588,39.786698,0.491756,39.763033,0.444139,0.496873,0.409120,0.450226,0.439298,0.388469,0.404781
min,0.000000,31.000000,56.000000,43.000000,0.000000,-1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,3.300000,79.000000,119.000000,103.000000,0.000000,5.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,3.800000,91.000000,133.000000,114.000000,0.000000,16.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,4.300000,122.000000,165.000000,136.000000,1.000000,37.250000,1.000000,1.000000,0.000000,1.000000,1.000000,0.000000,0.000000
max,5.000000,212.000000,331.000000,271.000000,1.000000,239.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [11]:
# Address empty values
print('Proportion of empty values in dataset:')
print(scraped_jobs_data.isnull().sum() / scraped_jobs_data.count())

Proportion of empty values in dataset:
Job Title            0.0
Salary Estimate      0.0
Job Description      0.0
Rating               0.0
Company Name         0.0
Location             0.0
Headquarters         0.0
Size                 0.0
Type of ownership    0.0
Industry             0.0
Sector               0.0
Revenue              0.0
min_salary           0.0
max_salary           0.0
avg_salary           0.0
job_state            0.0
same_state           0.0
company_age          0.0
python               0.0
excel                0.0
hadoop               0.0
spark                0.0
aws                  0.0
tableau              0.0
big_data             0.0
job_simp             0.0
seniority            0.0
dtype: float64


### **Questions to ask**


In our analysis of the data science job market, several key questions come to mind:

> 1.   Which sectors are hiring the most data science professionals at the moment?
>
> 2.   Which skills do hiring companies require from different data science professionals?
>
> 3.   Which job titles for data science professionals tend to have the highest estimated salary?
>
> 4.   Does the size of the company influence the estimated salary?
>



